In [1]:
#PreProcessing
import pickle
import sys

#this function remove stop word from text
def Remove_Stopwords(text_tokenized):
    text_without_sw=[token for token in text_tokenized if token.lower() not in stop_words]
    return text_without_sw

#this function Remove_Punctuation from text
def Remove_Punctuation(text_tokenized_wsw):
    text_without_pun=[token for token in text_tokenized_wsw if token not in punctuations]
    return text_without_pun

#this Function coverts letters into lower.
def casefolding(text):
    text_casefolded=[]
    for token in text:
        text_casefolded.append(token.lower())
    return text_casefolded


#this function stem the words by porter stemmer
def stemming_words(text):
    stemmed_words=[]
    for token in text:
        stemmed_words.append(porter.stem(token))
    return stemmed_words

#this word tokenize the text into words and replace - with space
def word_tokenization(data):
    data=data.lower()
    
    characters="abcdefghijklmnopqrstuvwxyz"
    
    special_character=['\n',' ']
    
    for i in data:
        if i == '-':
            data=data.replace(i," ")
        if i not in characters:
            if i not in special_character:
                data=data.replace(i,"")
    data=data.split()
    return(data)
#this function write inverted_index_to_file
def Write_inverted_index_to_file(inverted_index):
    sorted_inverted_index = {}
    sorted_inverted_index_keys = sorted(inverted_index.keys())  

    for term in sorted_inverted_index_keys:
        sorted_inverted_index[term] = inverted_index[term]
    file = open("inverted_index.pkl", "wb")
    pickle.dump(sorted_inverted_index, file)
    file.close()
    
    
#check the Validity of the Query
def check_query_valid_or_not(Query,no_of_terms):
    if no_of_terms==1:
        
        if len(Query)==2:
            if(str(Query[0]).lower()=="not"):
                return 1
            else:
                if str(Query[0]).lower()!="not":
                    return 0
    
    
    if no_of_terms==2:
        if(str(Query[0]).lower()=="not"):
            if ((str(Query[2]).lower()=="and") or (str(Query[2]).lower()=="or")):
                return 1
            else:
                return 0
        if(str(Query[0]).lower()!="not"):
            if ((str(Query[1]).lower()=="and") or (str(Query[1]).lower()=="or")):
                return 1
            else:
                return 0
    if no_of_terms==3:
        
        if(str(Query[0]).lower()=="not"):
            if ((str(Query[2]).lower()=="and") or (str(Query[2]).lower()=="or")):
                if(str(Query[3]).lower()=="not"):
                    if ((str(Query[5]).lower()=="and") or (str(Query[5]).lower()=="or")):
                        return 1
                    else:
                        return 0
                else:
                    if(str(Query[3]).lower()!="not"):
                        if ((str(Query[4]).lower()=="and") or (str(Query[4]).lower()=="or")):
                            return 1
                        else:
                            return 0      
            else:
                return 0
        else:
             if(str(Query[0]).lower()!="not"):
                    if ((str(Query[1]).lower()=="and") or (str(Query[1]).lower()=="or")):
                        if(str(Query[2]).lower()=="not"):
                            if ((str(Query[4]).lower()=="and") or (str(Query[4]).lower()=="or")):
                                return 1
                            else:
                                return 0
                        else:
                            if(str(Query[2]).lower()!="not"):
                                if ((str(Query[3]).lower()=="and") or (str(Query[3]).lower()=="or")):
                                       return 1
                                else:
                                    return 0
                                
                            
                    else:
                        return 0
                         

In [2]:
import nltk
from nltk.stem import PorterStemmer
porter = PorterStemmer()

#open file of stop_words
file=open("C:\\Users\\Yougesh\\CS317-IR Dataset for A1\\Stopword-List.txt",'r')
#read file of stop_words

stop_words=file.read()

#punctuations tobe removed
punctuations = '''!()-[]{};:'‘"\,<>./?@#$%^&*“’”_~'''


inverted_index=dict()

list_t=""
for x in range(1,51):
    #file opening one by one
    file=open("C:\\Users\\Yougesh\\CS317-IR Dataset for A1\\ShortStories\\"+str(x)+".txt",'r',encoding="utf8")
    
    data=file.read()
    #call the word_tokenization function
    tokenized_text=word_tokenization(data)
    #call the function to remove stop words
    text_without_sw=Remove_Stopwords(tokenized_text)
    #call the function to remove punctuation.
    text_cleaned=Remove_Punctuation(text_without_sw)
   #call the function of casefolding
    text_casefolded=casefolding(stemmed_words)
    #call the function to stem
    stemmed_words=stemming_words(text_cleaned)
    
   #Building Inverted_Index
    if x==1:
        for token in stemmed_words:
            inverted_index[token]=[]
            inverted_index[token].append(x)
    else:
        list_keys=inverted_index.keys()
        for token in stemmed_words:
            if token not in list_keys:
                inverted_index[token]=[]
        for token in stemmed_words:
                if token in list_keys:
                    if x not in inverted_index.get(token):
                        inverted_index[token].append(x)

#call the function to write an inverted index to file
Write_inverted_index_to_file(inverted_index)

NameError: name 'stemmed_words' is not defined

In [10]:
#Query processing.

#Loading invertedIndex in dictionary_object.
file = open("inverted_index.pkl", "rb")
inverted_index = pickle.load(file)
#operators tobe used in query
Operators=['not','and','or']
#list docID contains an list of all docID's
docID=[]
for i in range(1,51):
    docID.append(i)

Query=input("Enter the Query:")

Query=Query.split()

final_res=[]
#finding the no of terms in the Query
no_of_terms=[term for term in Query if term not in Operators]
#check wheter Query is valid or not
if check_query_valid_or_not(Query,len(no_of_terms))==0:
    print("Invalid Query Syntax")
    sys.exit()
    
#if there is one term in the Query
if len(no_of_terms)==1:
    res=[]
    for term in no_of_terms:
        if porter.stem(term.lower()) in inverted_index:
            res=inverted_index.get(porter.stem(term.lower()))
        #if Query contain not
        if len(Query)==2:
                res=[term for term in docID if term not in res]
    final_res=res

    
#If there is two terms in the Query
if len(no_of_terms)==2:
    res1=[]
    res2=[]
    operator_index=0
    #if there is not with the first term
    if(str(Query[0]).lower()=="not"):
        #if the term in the inverted_index
        if porter.stem(Query[1].lower()) in inverted_index:
                res1=inverted_index.get(porter.stem(Query[1].lower()))
        res1=[term for term in docID if term not in res1]
        operator_index=2
    elif (str(Query[0]).lower()!="not"): 
        if porter.stem(Query[0].lower()) in inverted_index:
                res1=inverted_index.get(porter.stem(Query[0].lower()))
        operator_index=1
    if(str(Query[operator_index+1]).lower()=="not"):
        if porter.stem(Query[operator_index+2].lower()) in inverted_index:
            res2=inverted_index.get(porter.stem(Query[operator_index+2].lower()))
        res2=[term for term in docID if term not in res2]
    elif(str(Query[operator_index+1]).lower()!="not"):
        if porter.stem(Query[operator_index+1].lower()) in inverted_index:
            res2=inverted_index.get(porter.stem(Query[operator_index+1].lower()))
    #if Query Contain and operator in between
    if(str(Query[operator_index]).lower()=="and"):
        #if res1 has less posting list
        if(len(res1)<len(res2)):
            final_res=[term for term in res1 if term in res2]
        else:
            final_res=[term for term in res2 if term in res1]
    if(str(Query[operator_index]).lower()=="or"):
        final_res=list(set().union(res1,res2))
        
#Query Processing when no od terms are three.
if len(no_of_terms)==3:
    res1=[]
    res2=[]
    res3=[]
    operator_index1=0
    operator_index2=0
    if(str(Query[0]).lower()=="not"):
        if porter.stem(Query[1].lower()) in inverted_index:
            res1=inverted_index.get(porter.stem(Query[1].lower()))
        res1=[term for term in docID if term not in res1]
        operator_index1=2
    elif (str(Query[0]).lower()!="not"):
        if porter.stem(Query[0].lower()) in inverted_index:
                res1=inverted_index.get(porter.stem(Query[0].lower()))
        operator_index1=1
    if(str(Query[operator_index1+1]).lower()=="not"):
        if porter.stem(Query[operator_index1+2].lower()) in inverted_index:
            res2=inverted_index.get(porter.stem(Query[operator_index1+2].lower()))
        res2=[term for term in docID if term not in res2]
        operator_index2=operator_index1+3
    elif(str(Query[operator_index1+1]).lower()!="not"):  
        if porter.stem(Query[operator_index1+1].lower()) in inverted_index:
            res2=inverted_index.get(porter.stem(Query[operator_index1+1].lower())) 
        operator_index2=operator_index1+2
    if(str(Query[operator_index2+1]).lower()=="not"):
        if porter.stem(Query[operator_index2+2].lower()) in inverted_index:
                res3=inverted_index.get(porter.stem(Query[operator_index2+2].lower()))
        res3=[term for term in docID if term not in res3]
    elif(str(Query[operator_index2+1]).lower()!="not"):
        if porter.stem(Query[operator_index2+1].lower()) in inverted_index:
            res3=inverted_index.get(porter.stem(Query[operator_index2+1].lower()))
    
    if(str(Query[operator_index1]).lower()=="and"):
        if(len(res1)<len(res2)):
            res2=[term for term in res1 if term in res2]
        else:
            res2=[term for term in res2 if term in res1]
        if (str(Query[operator_index2]).lower()=="and"):
            if(len(res2)<len(res3)):
                final_res=[term for term in res2 if term in res3]
            else:
                final_res=[term for term in res3 if term in res2]
        else:
            final_res=list(set().union(res2,res3))
    if(str(Query[operator_index1]).lower()=="or"):
        res2=list(set().union(res1,res2))
        if (str(Query[operator_index2]).lower()=="and"):
            if(len(res2)<len(res3)):
                final_res=[term for term in res2 if term in res3]
            else:
                final_res=[term for term in res3 if term in res2]
        else:
             final_res=list(set().union(res2,res3))
    
print("Result-Set:"+str(final_res))

Enter the Query:god and man and love
Result-Set:[1, 2, 4, 7, 8, 9, 14, 16, 19, 22, 23, 24, 25, 26, 27, 28, 31, 38, 46]
